In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = "crowdfunding_dataset.csv"  # Correct file path here
data = pd.read_csv(file_path)

# # Check dataset columns
# print("Columns in dataset:", data.columns)

# # Check class distribution
# print("\nSuccess Label Counts:\n", data["success"].value_counts())

# # Ensure 'success' column is binary (0 = failed, 1 = successful)
# if "success" not in data.columns:
#     raise ValueError("Column 'success' not found in dataset!")

# One-hot encode categorical features
data_encoded = pd.get_dummies(data, columns=["parent_category", "sub_category", "country"], drop_first=True)

# Define features and target
X = data_encoded.drop(columns=["success"])
y = data_encoded["success"]

# Stratified splitting to maintain class ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Handling class imbalance (choose one method)

# ✅ Method 1: Undersampling (Reduce '0' class)
# rus = RandomUnderSampler(random_state=42)
# X_train, y_train = rus.fit_resample(X_train, y_train)

# ✅ Method 2: Oversampling (Increase '1' class)
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Train AdaBoost model
ab_model = AdaBoostClassifier(n_estimators=3, random_state=42)
ab_model.fit(X_train, y_train)



# Predictions
y_pred_ab = ab_model.predict(X_test)

# Function to evaluate models
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"\n{model_name} Metrics:")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%\n")
    return accuracy

# Evaluate model
ab_accuracy = evaluate_model(y_test, y_pred_ab, "AdaBoost")

# Save the trained models
joblib.dump(ab_model, "adaboost_model.pkl")

print("Model training complete. Models saved as 'adaboost_model.pkl'.")



C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



AdaBoost Metrics:
Accuracy: 92.02%
Precision: 77.54%
Recall: 95.42%
F1 Score: 85.56%

Model training complete. Models saved as 'adaboost_model.pkl'.
